In [1]:
# app.py
# ------------------------------------------------------------
# One‑Page Supermarket Product Sales Dashboard  (Dash + Plotly + Dash‑Bootstrap‑Components)
# ------------------------------------------------------------

import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX

import dash
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc, dash_table, Input, Output, State
import plotly.graph_objects as go

# ------------------------------------------------------------
# 1) SYNTHETIC DATA (Product Sales)
# ------------------------------------------------------------
kpi_dict = {
    "Total Sales (USD k)": 1420,
    "YTD Growth": "12.5%",
    "New Customers YTD": 865,
    "Top-Selling Products": 5,
}

# Monthly sales for a popular product (last 24 months)
np.random.seed(123)
hist_idx = pd.date_range("2022-01-31", periods=24, freq="ME")
hist_vals = np.random.randint(80, 200, len(hist_idx))
sales_series = pd.Series(hist_vals, index=hist_idx)

# SARIMAX forecast (next 6 months)
model = SARIMAX(sales_series, order=(1, 0, 1),
                seasonal_order=(1, 0, 0, 12),
                enforce_stationarity=False, enforce_invertibility=False)
fit = model.fit(disp=False)
steps = 6
forecast_res = fit.get_forecast(steps=steps)
fc_idx = pd.date_range(sales_series.index[-1] + relativedelta(months=1),
                       periods=steps, freq="ME")
fc_vals = forecast_res.predicted_mean
fc_ci = forecast_res.conf_int(alpha=0.2)
fc_ci.index = fc_idx
low_demand_flag = fc_vals < 100

# Adoption curve (new product sales ramp-up)
adopt_idx = pd.date_range("2024-03-31", periods=6, freq="QE")
adopt_vals = [50, 120, 190, 260, 340, 410]

# Top Customer Purchases Table
top_customers = ["Smith Mart", "Green's Grocery", "Urban Basket", "QuickBuy",
                 "Happy Fresh", "Budget Box", "Nova Foods", "SmartChoice"]
quarters = pd.date_range("2024-03-31", periods=4, freq="QE")
records = []
for cust in top_customers:
    start_q = np.random.randint(0, len(quarters))
    for i, q in enumerate(quarters):
        purchases = 0
        if i >= start_q:
            purchases = np.random.randint(2000, 8000)
        records.append({"Customer": cust,
                        "Quarter": q.date(),
                        "PurchaseVolume": purchases})
df_cust = pd.DataFrame(records)
first_orders = (df_cust[df_cust["PurchaseVolume"] > 0]
                .groupby("Customer")["Quarter"]
                .min()
                .reset_index()
                .rename(columns={"Quarter": "FirstOrder"}))
cutoff = quarters[1].date()
first_orders["EarlyAdopter"] = np.where(first_orders["FirstOrder"] <= cutoff,
                                        "YES", "NO")

# ------------------------------------------------------------
# 2) PLOTLY FIGURES
# ------------------------------------------------------------
fig_sales = go.Figure()
fig_sales.add_trace(go.Scatter(x=sales_series.index, y=sales_series,
                               mode="lines+markers", name="Historical Sales"))
fig_sales.add_trace(go.Scatter(x=fc_idx, y=fc_vals,
                               mode="lines+markers", line=dict(dash="dash"), name="Forecast"))
fig_sales.add_trace(go.Scatter(x=fc_idx, y=fc_ci.iloc[:, 0],
                               mode="lines", line=dict(width=0), showlegend=False))
fig_sales.add_trace(go.Scatter(x=fc_idx, y=fc_ci.iloc[:, 1],
                               mode="lines", line=dict(width=0),
                               fill="tonexty", fillcolor="rgba(0,100,200,0.15)", name="80% CI"))
fig_sales.update_layout(height=280, margin=dict(l=30, r=10, t=25, b=25),
                        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_adopt = go.Figure()
fig_adopt.add_trace(go.Scatter(x=adopt_idx, y=adopt_vals, mode="lines+markers"))
fig_adopt.update_layout(height=280, margin=dict(l=30, r=10, t=25, b=25))

# ------------------------------------------------------------
# 3) DASH LAYOUT
# ------------------------------------------------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

kpi_cards = dbc.Row([
    dbc.Col(dbc.Card(
        dbc.CardBody([
            html.P(label, className="text-muted small"),
            html.H4(value, className="card-title")
        ]), className="text-center"),
        md=3, sm=6) for label, value in kpi_dict.items()
])

forecast_table = pd.DataFrame({
    "Month": fc_idx.strftime("%Y-%m"),
    "Forecast Sales (k USD)": fc_vals.round(1),
    "Low Demand?": np.where(low_demand_flag, "YES", "NO")
})

app.layout = dbc.Container(fluid=True, children=[
    html.H3("Supermarket Product Sales Dashboard", className="mt-3 mb-4"),

    kpi_cards,
    html.Hr(),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("6-Month Sales Forecast"),
            dbc.CardBody(dcc.Graph(figure=fig_sales, config={"displayModeBar": False}))
        ]), md=6),

        dbc.Col(dbc.Card([
            dbc.CardHeader("New Product Adoption Trend"),
            dbc.CardBody(dcc.Graph(figure=fig_adopt, config={"displayModeBar": False}))
        ]), md=6)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Early-Adopter Customers"),
            dbc.CardBody(dash_table.DataTable(
                data=first_orders.to_dict("records"),
                columns=[{"name": c, "id": c} for c in first_orders.columns],
                style_cell={"fontSize": 12, "padding": "4px"},
                style_header={"backgroundColor": "#f2f2f2", "fontWeight": "bold"},
                page_size=8
            ))
        ]), md=6),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Recommended Sales Actions"),
            dbc.CardBody([
                dbc.Button("Toggle", id="toggle-btn", size="sm", className="mb-2"),
                dbc.Collapse(dbc.ListGroup([
                    dbc.ListGroupItem("Send loyalty coupons to early adopters."),
                    dbc.ListGroupItem("Prepare inventory before low demand forecast months."),
                    dbc.ListGroupItem("Launch marketing campaigns during Q2 and Q3."),
                    dbc.ListGroupItem("Track regional buying trends for promotions.")
                ]), id="actions-collapse", is_open=True)
            ])
        ]), md=6)
    ]),

    html.Hr(className="mt-4 mb-2"),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("6-Month Sales Forecast Table"),
            dbc.CardBody(dash_table.DataTable(
                data=forecast_table.to_dict("records"),
                columns=[{"name": c, "id": c} for c in forecast_table.columns],
                style_cell={"fontSize": 12, "padding": "4px"},
                style_header={"backgroundColor": "#f9f9f9", "fontWeight": "bold"},
                page_size=6
            ))
        ]), md=12)
    ], className="mb-4")
], style={"maxWidth": "1300px"})

# ------------------------------------------------------------
# 4) CALLBACKS
# ------------------------------------------------------------
@app.callback(
    Output("actions-collapse", "is_open"),
    Input("toggle-btn", "n_clicks"),
    State("actions-collapse", "is_open"),
    prevent_initial_call=True
)
def toggle_actions(n, is_open):
    return not is_open if n else is_open

# ------------------------------------------------------------
# 5) MAIN
# ------------------------------------------------------------
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=8053, debug=True)
